<a href="https://colab.research.google.com/github/NNFLgroup31/TemporalModelling-LSTM/blob/master/rnn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import inspect
import torch
import torch.nn as nn
from  torch.autograd import Variable
import torch.nn.utils.rnn as rnn_utils 
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter as summary
import _pickle as pickle
import os
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class Model(nn.Module):
  
  def __init__(self,device, n_input=9 * 6 + 1, n_classes=23, batch_size=500, max_obs=26,
                 n_layers=2, dropout_keep_prob=.5, adam_lr=1e-3, adam_b1=0.9, adam_b2=0.999, adam_eps=1e-8,
                 fc_w_stddev=0.1, fc_b_offset=0.1, n_cell_per_input=1, gpu=None):
    super(Model,self).__init__()
    self.args=inspect.getargvalues(inspect.currentframe()).locals
    del self.args["self"]
    self.n_classes=n_classes
    self.n_layers=n_layers
    self.batch_size=batch_size
    self.wstd=fc_w_stddev
    self.b=fc_b_offset
    ##input model###
    self.n_rnn_cells = n_rnn_cells = n_cell_per_input * n_input
    self.layer1=nn.LSTM(n_input,hidden_size=n_rnn_cells,num_layers=self.n_layers,dropout=dropout_keep_prob,batch_first=True)
    # self.layer2=nn.Softmax(dim=1)
    self.opt=optim.Adam(self.parameters(),lr=adam_lr,betas=(adam_b1,adam_b2),eps=adam_eps)
    self.writer=summary()
    self.max_obs=max_obs
    self.device=device
  def forward(self,X,hidden):
    out,hidden=self.layer1(X,hidden)
  
    out=out.reshape(-1,out.size(2))
    W=torch.normal(mean=0,std=self.wstd,size=(int(out.size(1)),self.n_classes))
    b=self.b*torch.ones(self.n_classes)
    out=torch.matmul(out,W)+b
    out=out.reshape(self.batch_size,-1,self.n_classes)
    self.out=out
    self.hidden=hidden
    return out,hidden
 
 
  def lossf(self,out,target,obs,train=True):
    out=nn.functional.softmax(out,dim=2)
    out=out.reshape(-1,out.size(2))
    self.mask=mask=self._sequence_mask(obs,target.size(1)).to((self.device))
  
    target=target.reshape(-1,target.size(2))
    loss=-target*torch.log(out)
    mask=mask.reshape(-1,1)
    mask=mask.expand_as(loss)
    loss=mask.double()*loss
    loss=torch.sum(loss,1)
    a=torch.sum(loss>0)
    loss=torch.sum(loss)/a
    self.total_samples=a
    if train:
      self.opt.zero_grad()
      loss.backward()
      self.opt.step()
  
    self.writer.add_scalar('cross_entropy',loss.item(),None)
    return loss
  
  def repack_hidden(self):
      a,b=self.hidden
      a=a.detach_()
      b=b.detach_()
      self.hidden=(a,b)
      return self.hidden

  def evaluation(self,out,target):
    self.probabilities=probs=nn.functional.softmax(out,dim=2)

    #Evaluate model
    predicted=torch.argmax(out,dim=2)
    targets=torch.argmax(target,dim=2)
    correct_pred=torch.tensor(predicted==targets,device=None)
    mask_correct_pred=torch.bitwise_and(correct_pred,self.mask)
    self.accuracy=accuracy=torch.sum(mask_correct_pred.float())/self.total_samples
    self.writer.add_scalar('accuracy',accuracy)

    self.probs_list=probs_list=torch.reshape(probs,(-1,self.n_classes))
    predicted_list=torch.reshape(predicted,(-1,1))
    targets_list=torch.reshape(targets,(-1,1))

    mask_list=torch.reshape(self.mask,(-1,1))
    one_hot_targets=torch.zeros((predicted_list.size(0),self.n_classes),dtype=torch.float32,device=None)
    one_hot_targets.scatter_(1,targets_list,1)
    scores=torch.masked_select(probs_list,torch.tensor(one_hot_targets==1))

    obs_list=torch.arange(0,self.max_obs).expand(self.batch_size,-1)
    probs_matrix_mask=mask_list.expand(-1,self.n_classes)
    
    self.scores=torch.masked_select(probs_list,probs_matrix_mask)
    self.targets=torch.masked_select(targets_list,probs_matrix_mask)
    

    target_=torch.unsqueeze(target,1)
    self.writer.add_images('targets',target_,dataformats='NCHW')

    probs_=torch.unsqueeze(probs,1)
    self.writer.add_images('probabilities',probs_,dataformats='NCHW')

    logits_=torch.unsqueeze(out,1)
    self.writer.add_images('logits_',target_,dataformats='NCHW')
    return None
  def _sequence_mask(self,sequence_length, max_len=None):
      if max_len is None:
          max_len = sequence_length.data.max()
      batch_size = sequence_length.size(0)
      seq_range = torch.arange(0, max_len ).long()
      seq_range_expand = seq_range.unsqueeze(0).expand(batch_size, max_len)
      seq_range_expand = Variable(seq_range_expand)
      if sequence_length.is_cuda:
          seq_range_expand = seq_range_expand.cuda()
      seq_length_expand = (sequence_length.unsqueeze(1)
                          .expand_as(seq_range_expand))
      return seq_range_expand < seq_length_expand


    
  def intial_hiddenstate(self):
     weight = next(self.parameters()).data
     return (Variable(weight.new(self.n_layers,self.batch_size, self.n_rnn_cells).zero_()).to(str(self.device)),
                Variable(weight.new(self.n_layers,self.batch_size, self.n_rnn_cells).zero_()).to(str(self.device)))
     ###add gpu compatibility
        #  if self.hparams.on_gpu:
        #     hidden_a = hidden_a.cuda()
        #     hidden_b = hidden_b.cuda()

        # hidden_a = Variable(hidden_a)
        # hidden_b = Variable(hidden_b)

def main():
      model=Model(device=device)
      # path='gdrive/My Drive/lstm/rundir'
      # pickle.dump(model.args,open(os.path.join(path,'args.pkl'),"wb"))
      # open(os.path.join(path, "args.txt"), "w").write(str(model.args))
      #  writer=summary("gdrive/My Drive/lstm/rundir")
      #  writer=model.writer
      # writer.close()
      
if __name__ == '__main__':
    main()

In [ ]:
!git clone https://github.com/NNFLgroup31/TemporalModelling-LSTM.git

Cloning into 'TemporalModelling-LSTM'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.
